Read CSV data with Pandas.

In [1]:
import pandas as pd

# team stats
team_summaries = pd.read_csv('nba_data/stats_archive/Team Summaries.csv')
# team_totals = pd.read_csv('nba_data/stats_archive/Team Totals.csv')
team_per100 = pd.read_csv('nba_data/stats_archive/Team Stats Per 100 Poss.csv')

# player stats
# player_totals = pd.read_csv('nba_data/stats_archive/Player Totals.csv')
player_shooting = pd.read_csv('nba_data/stats_archive/Player Shooting.csv')
player_play_by_play = pd.read_csv('nba_data/stats_archive/Player Play By Play.csv')
player_per100 = pd.read_csv('nba_data/stats_archive/Per 100 Poss.csv')

# games
games = pd.read_csv('nba_data/game_archive/games.csv')
teamids = pd.read_csv('nba_data/game_archive/teams.csv')


Remove unnecessary features.

In [2]:
# games
teamids['TEAM_NAME'] = teamids['CITY'] + ' ' + teamids['NICKNAME']
teamids = teamids[['TEAM_ID','TEAM_NAME','ABBREVIATION']]

games = games[['GAME_DATE_EST','GAME_ID','HOME_TEAM_ID','VISITOR_TEAM_ID','SEASON','HOME_TEAM_WINS','PTS_home','PTS_away']]

In [3]:
# team stats
team_stats = team_summaries.merge(team_per100)
team_stats.drop(columns=['playoffs','arena','attend','lg'], inplace=True)

# player stats
player_stats = player_shooting.merge(player_play_by_play).merge(player_per100)
player_stats.drop(columns=['birth_year','pos','lg'], inplace=True)

In [4]:
team_stats

,season,team,abbreviation,age,w,l,pw,pl,mov,sos,...,ft_percent,orb_per_100_poss,drb_per_100_poss,trb_per_100_poss,ast_per_100_poss,stl_per_100_poss,blk_per_100_poss,tov_per_100_poss,pf_per_100_poss,pts_per_100_poss
0,2024,Atlanta Hawks,ATL,26.3,34.0,40.0,34.0,40.0,-1.36,-0.17,...,0.802,12.7,31.7,44.3,26.2,7.5,4.6,13.1,18.3,118.2
1,2024,Boston Celtics,BOS,28.3,58.0,16.0,59.0,15.0,11.38,-0.33,...,0.809,10.9,36.5,47.4,27.2,6.8,6.6,12.3,16.7,123.3
2,2024,Brooklyn Nets,BRK,26.3,29.0,46.0,32.0,43.0,-2.37,-0.08,...,0.757,11.7,33.1,44.8,26.7,7.2,5.1,13.2,19.2,113.7
3,2024,Chicago Bulls,CHI,28.0,36.0,39.0,34.0,41.0,-1.48,-0.25,...,0.794,11.2,33.4,44.6,25.7,7.8,5.0,12.5,19.2,114.7
4,2024,Charlotte Hornets,CHO,25.2,18.0,56.0,16.0,58.0,-10.55,-0.16,...,0.779,9.9,31.7,41.6,25.2,6.9,4.5,14.1,18.6,109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397,1974,San Antonio Spurs,SAA,24.6,45.0,39.0,45.0,39.0,0.87,-0.13,...,0.778,15.3,32.8,48.2,22.4,7.4,2.8,16.5,27.3,101.7
1398,1974,San Diego Conquistadors,SDA,25.1,37.0,47.0,36.0,48.0,-2.54,0.26,...,0.753,15.5,29.6,45.2,21.5,7.5,5.6,14.3,17.8,102.9
1399,1974,Seattle SuperSonics,SEA,26.0,36.0,46.0,34.0,48.0,-2.55,0.26,...,0.767,14.6,29.8,44.4,23.2,7.6,3.2,17.9,22.9,96.8
1400,1974,Utah Stars,UTS,27.8,51.0,33.0,43.0,41.0,0.36,-0.08,...,0.789,13.8,31.6,45.4,22.4,8.4,4.2,17.1,23.2,105.6


Create player efficiency indices.

𝐼NBA=𝑁pts+𝑁rbs+𝑁asts+𝑁stls+𝑁blcks−(𝑁msFld+𝑁msFT+𝑁tos)

In [5]:
player_stats['eff_ind'] = (player_stats['pts_per_100_poss'] +
                           player_stats['trb_per_100_poss'] +
                           player_stats['ast_per_100_poss'] +
                           player_stats['stl_per_100_poss'] +
                           player_stats['blk_per_100_poss'] -
                           (player_stats['tov_per_100_poss'] +
                            player_stats['fga_per_100_poss'] - player_stats['fg_per_100_poss'] +
                            player_stats['fta_per_100_poss'] - player_stats['ft_per_100_poss']))

player_stats

,seas_id,season,player_id,player,age,experience,tm,g,mp,fg_percent,...,trb_per_100_poss,ast_per_100_poss,stl_per_100_poss,blk_per_100_poss,tov_per_100_poss,pf_per_100_poss,pts_per_100_poss,o_rtg,d_rtg,eff_ind
0,31136,2024,5025,A.J. Green,24,2,MIL,50,509,0.427,...,5.3,2.5,0.6,0.3,0.8,4.1,20.8,124.0,120.0,19.4
1,31137,2024,5026,A.J. Lawson,23,2,DAL,36,254,0.480,...,6.8,2.6,1.9,0.6,1.9,3.8,22.7,111.0,117.0,21.6
2,31138,2024,5027,AJ Griffin,20,2,ATL,18,132,0.289,...,5.1,1.5,0.4,0.4,2.2,2.2,13.4,82.0,124.0,6.9
3,31139,2024,4219,Aaron Gordon,28,10,DEN,68,2145,0.551,...,10.3,5.3,1.2,0.9,2.3,3.1,21.8,123.0,115.0,28.3
4,31140,2024,4582,Aaron Holiday,27,6,HOU,71,1168,0.445,...,4.9,5.4,1.6,0.2,2.0,4.6,19.6,117.0,115.0,20.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16763,15656,1997,2632,Willie Burton,28,6,ATL,24,380,0.336,...,6.0,1.6,1.2,0.4,3.8,8.0,21.5,100.0,106.0,14.1
16764,15657,1997,2278,Xavier McDaniel,33,11,NJN,62,1170,0.389,...,14.1,2.9,1.6,0.8,3.1,6.4,15.4,97.0,109.0,20.9
16765,15658,1997,2890,Yinka Dare,24,3,NJN,41,313,0.352,...,13.6,0.5,0.7,4.6,3.5,8.5,9.5,83.0,107.0,16.7
16766,15567,1997,2540,Šarūnas Marčiulionis,32,7,DEN,17,255,0.376,...,6.1,5.1,2.4,0.2,8.1,7.7,23.6,85.0,112.0,15.0


Create team efficiency indices.

In [6]:
#create the team efficeny index by summing the players by season and team 
team_efficiency_index = player_stats.groupby(['tm', 'season'])['eff_ind'].sum().reset_index(name='team_efficiency_index')
#merging the team efficeny index to team_stats
team_stats = team_stats.merge(team_efficiency_index, left_on=['abbreviation', 'season'], right_on=['tm', 'season'], how='left')
team_stats.drop(columns = ["tm"], inplace=True)
team_stats = team_stats.dropna()
team_stats

,season,team,abbreviation,age,w,l,pw,pl,mov,sos,...,orb_per_100_poss,drb_per_100_poss,trb_per_100_poss,ast_per_100_poss,stl_per_100_poss,blk_per_100_poss,tov_per_100_poss,pf_per_100_poss,pts_per_100_poss,team_efficiency_index
0,2024,Atlanta Hawks,ATL,26.3,34.0,40.0,34.0,40.0,-1.36,-0.17,...,12.7,31.7,44.3,26.2,7.5,4.6,13.1,18.3,118.2,392.6
1,2024,Boston Celtics,BOS,28.3,58.0,16.0,59.0,15.0,11.38,-0.33,...,10.9,36.5,47.4,27.2,6.8,6.6,12.3,16.7,123.3,566.3
2,2024,Brooklyn Nets,BRK,26.3,29.0,46.0,32.0,43.0,-2.37,-0.08,...,11.7,33.1,44.8,26.7,7.2,5.1,13.2,19.2,113.7,505.0
3,2024,Chicago Bulls,CHI,28.0,36.0,39.0,34.0,41.0,-1.48,-0.25,...,11.2,33.4,44.6,25.7,7.8,5.0,12.5,19.2,114.7,467.2
4,2024,Charlotte Hornets,CHO,25.2,18.0,56.0,16.0,58.0,-10.55,-0.16,...,9.9,31.7,41.6,25.2,6.9,4.5,14.1,18.6,109.0,528.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2001,Toronto Raptors,TOR,29.9,47.0,35.0,48.0,34.0,2.26,-0.57,...,14.8,33.4,48.2,26.5,7.9,6.9,14.3,23.1,105.9,416.2
713,2001,Utah Jazz,UTA,32.0,53.0,29.0,55.0,27.0,4.70,0.31,...,12.7,32.2,45.0,28.5,8.9,6.3,17.5,28.5,107.6,315.3
714,2001,Vancouver Grizzlies,VAN,26.2,23.0,59.0,25.0,57.0,-5.73,0.79,...,11.8,32.2,44.0,25.1,7.8,4.8,17.1,22.9,99.6,303.5
715,2001,Washington Wizards,WAS,27.0,19.0,63.0,23.0,59.0,-6.67,-0.08,...,13.4,31.2,44.6,21.7,8.3,5.0,18.3,25.2,100.6,446.9


In [7]:
final_df = team_stats[(team_stats['season'] >= 2018) & (team_stats['season'] <= 2021)]
final_df

,season,team,abbreviation,age,w,l,pw,pl,mov,sos,...,orb_per_100_poss,drb_per_100_poss,trb_per_100_poss,ast_per_100_poss,stl_per_100_poss,blk_per_100_poss,tov_per_100_poss,pf_per_100_poss,pts_per_100_poss,team_efficiency_index
90,2021,Atlanta Hawks,ATL,25.4,41.0,31.0,41.0,31.0,2.32,-0.18,...,10.7,35.7,46.4,24.5,7.1,4.8,13.5,19.7,115.7,420.7
91,2021,Boston Celtics,BOS,25.1,36.0,36.0,39.0,33.0,1.46,-0.14,...,10.8,34.0,44.8,23.7,7.8,5.4,14.2,20.7,114.0,501.2
92,2021,Brooklyn Nets,BRK,28.2,48.0,24.0,46.0,26.0,4.50,-0.27,...,8.9,35.5,44.3,26.7,6.7,5.3,13.5,19.0,118.3,631.4
93,2021,Chicago Bulls,CHI,25.6,31.0,41.0,34.0,38.0,-0.89,-0.05,...,9.7,35.5,45.1,26.9,6.7,4.2,15.2,19.0,111.1,525.1
94,2021,Charlotte Hornets,CHO,24.6,33.0,39.0,32.0,40.0,-1.93,-0.01,...,10.7,33.6,44.3,27.2,7.9,4.8,15.0,18.3,110.9,400.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,2018,Sacramento Kings,SAC,26.4,27.0,55.0,23.0,59.0,-6.99,0.39,...,9.9,33.0,42.9,22.6,8.2,4.4,14.4,21.0,103.7,393.4
206,2018,San Antonio Spurs,SAS,29.3,47.0,35.0,49.0,33.0,2.89,0.00,...,10.9,35.6,46.4,23.9,8.0,5.9,13.8,18.0,107.9,409.3
207,2018,Toronto Raptors,TOR,25.8,59.0,23.0,60.0,22.0,7.78,-0.49,...,9.9,34.9,44.8,24.8,7.8,6.2,13.6,22.2,113.8,427.4
208,2018,Utah Jazz,UTA,26.7,48.0,34.0,53.0,29.0,4.30,0.17,...,9.4,35.6,45.0,23.3,9.0,5.3,15.3,20.4,108.4,586.8


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = final_df.drop(['team', 'abbreviation', 'season', 'w'], axis=1)  # Features
y = final_df['w']  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'X train shape: {X_train.shape}')
print(f'X test shape: {X_test.shape}')

X train shape: (92, 47)
X test shape: (24, 47)


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

# Initialize models for regression
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest Regressor": RandomForestRegressor(),
    "KNN Regressor": KNeighborsRegressor(),
    "SVR": SVR(),
    "XGBoost Regressor": XGBRegressor(objective='reg:squarederror'),
    "Neural Network Regressor": MLPRegressor(max_iter=1000)
}

results = {}

for name, model in models.items():
    pipeline = make_pipeline(StandardScaler(), model)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    results[name] = mse
    print(f"{name} MSE: {mse:.4f}")

# Displaying all results
print("\nModel Mean Squared Errors:")
for name, mse in results.items():
    print(f"{name}: {mse:.4f}")


Linear Regression MSE: 0.0000
Random Forest Regressor MSE: 8.6299
KNN Regressor MSE: 21.7900
SVR MSE: 80.8252
XGBoost Regressor MSE: 9.7590
Neural Network Regressor MSE: 104.6257

Model Mean Squared Errors:
Linear Regression: 0.0000
Random Forest Regressor: 8.6299
KNN Regressor: 21.7900
SVR: 80.8252
XGBoost Regressor: 9.7590
Neural Network Regressor: 104.6257


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
